Importando as bibliotecas

In [4]:
import random
import requests
from collections import Counter
from math import log2

Inicializando o jogo da forca

In [5]:
class JogoDeForca:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro: ", r.status_code)

    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    print("Fim de jogo!")
                    return False
                else:
                    return []
        
    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra == palavra:
                print ("Ganhou!")
                return True
            else:
                self.vidas = 0
                print("Fim de jogo!")
                return False

Criar Jogador automático 

In [6]:
class JogadorAutomatico:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code == 200:
            self.vocabulario = str(r.content.decode()).split('\n')
        else:
            raise Exception("Erro ao carregar o vocabulário")

    def calcular_frequencias(self, palavras):
        letras = Counter("".join(palavras))
        total = sum(letras.values())
        return {letra: count / total for letra, count in letras.items()}

    def calcular_entropia(self, palavras, letra):
        total = len(palavras)
        if total == 0:
            return 0
        contem_letra = sum([1 for palavra in palavras if letra in palavra])
        p_contem = contem_letra / total
        p_nao_contem = 1 - p_contem
        entropia = 0
        if p_contem > 0:
            entropia -= p_contem * log2(p_contem)
        if p_nao_contem > 0:
            entropia -= p_nao_contem * log2(p_nao_contem)
        return entropia

    def escolher_proxima_letra(self, palavras_candidatas, letras_escolhidas):
        frequencias = self.calcular_frequencias(palavras_candidatas)
        letras_validas = [letra for letra in frequencias if letra not in letras_escolhidas]
        entropias = {letra: self.calcular_entropia(palavras_candidatas, letra) for letra in letras_validas}
        return max(entropias, key=entropias.get)

    def filtrar_palavras(self, palavras, letra, indices):
        if indices:
            return [palavra for palavra in palavras if all(palavra[i] == letra for i in indices)]
        return [palavra for palavra in palavras if letra not in palavra]

    def jogar(self, jogo):
        tentativas = set()
        tamanho_palavra = jogo.novo_jogo()
        palavras_candidatas = [p for p in self.vocabulario if len(p) == tamanho_palavra]

        while jogo.vidas > 0:
            letra = self.escolher_proxima_letra(palavras_candidatas, tentativas)
            tentativas.add(letra)
            indices = jogo.tentar_letra(letra)
            if indices is False:
                break
            palavras_candidatas = self.filtrar_palavras(palavras_candidatas, letra, indices)
            if len(palavras_candidatas) == 1:
                return jogo.tentar_palavra(palavras_candidatas[0])

        return False

Jogo Automático

In [7]:
# Inicializa as classes de jogo e jogador
jogo = JogoDeForca()
jogador = JogadorAutomatico()

Uma partida

In [8]:
# Executa uma partida automática
resultado = jogador.jogar(jogo)

# Exibe o resultado da partida
if resultado:
    print("O jogador venceu o jogo!")
else:
    print("O jogador perdeu o jogo!")
print("Vidas restantes:", jogo.vidas)


Fim de jogo!
O jogador perdeu o jogo!
Vidas restantes: 0


Multiplas Partidas

In [9]:
# Executa múltiplas partidas e calcula a taxa de vitória
num_jogos = 100
vitorias = 0

for _ in range(num_jogos):
    jogo = JogoDeForca()  # Reinicia o jogo
    if jogador.jogar(jogo):  # Joga a partida
        vitorias += 1

taxa_vitoria = (vitorias / num_jogos) * 100
print(f"Taxa de vitória do jogador automático: {taxa_vitoria:.2f}% em {num_jogos} jogos")


Ganhou!
Fim de jogo!
Ganhou!


KeyboardInterrupt: 